In [3]:
import gc

gc.collect()

import tensorflow  as tf
import tensorflow_hub as hub
import os
import matplotlib.pyplot as plt
import cv2
import numpy as np
import random
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow_hub as hub
import keras
import tensorflow_text as text

# Read Labels

In [4]:
csv_path = "slim_dataset.csv"
labels_df = pd.read_csv(csv_path)
print(labels_df)

           image_name  good  crack  corrossion
0         image_0.jpg  0.33   0.00        0.66
1         image_1.jpg  0.33   0.00        0.66
2         image_3.jpg  0.33   0.00        0.66
3         image_4.jpg  0.33   0.00        0.66
4         image_5.jpg  0.66   0.00        0.33
...               ...   ...    ...         ...
5289  2_image_993.jpg  0.33   0.66        0.00
5290  0_image_994.jpg  0.66   0.33        0.00
5291  1_image_994.jpg  0.66   0.33        0.00
5292  2_image_994.jpg  0.66   0.33        0.00
5293  2_image_998.jpg  1.00   0.00        0.00

[5294 rows x 4 columns]


# Make Predictions

In [5]:
model_path = "C:/Users/chuck/OneDrive/Desktop/Honors/models/Cell_classification/All_in_one/fine_tuned_model/"
loaded_model = tf.keras.models.load_model(model_path)

In [6]:
image_folder = "C:/Users/chuck/OneDrive/Desktop/Honors/propper_datasets/images"
column_names = labels_df.columns.tolist()
prediction_df = pd.DataFrame(columns=column_names)

In [7]:
def write_numbers(image, prediction, truth):
    # Define the font and font scale
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 0.5

    # Define the position and color for each number label
    positions = [(10, 20), (10, 50), (10, 90)]
    colors = [(0, 255, 0), (0, 0, 255), (255, 255, 0)]

    # Loop over each number and label, and write it onto the image
    for i, number in enumerate(prediction):
        label = f"{['good', 'crack', 'corrosion'][i]}: {number:.2f} ({truth[i]:.2f})"
        cv2.putText(image, label, positions[i], font, font_scale, (0,0,0), 2, cv2.LINE_AA)
        cv2.putText(image, label, positions[i], font, font_scale, colors[i], 1, cv2.LINE_AA)

    return image
    

In [8]:
for index, row in labels_df.iterrows():
    image_name = row["image_name"]
    image_path = image_folder + "/" + image_name

    img = cv2.imread(image_path)
    img = cv2.resize(img, (224, 224))  # resize to model input size
    img_array = np.array(img)/128 - 1     # convert to numpy array

    pred = loaded_model.predict(np.expand_dims(img_array, axis=0))[0]
    row = [image_name] + list(pred)
    prediction_df.loc[len(prediction_df)] = row
    print(f"...loading {round(100*index/len(labels_df), 2)}%...")

# prediction_df.to_csv("predictions.csv")

1/1 [==============================] - 8s 8s/step
...loading 0.0%...
1/1 [==============================] - 1s 1s/step
...loading 0.02%...
1/1 [==============================] - 1s 1s/step
...loading 0.04%...
1/1 [==============================] - 1s 1s/step
...loading 0.06%...
1/1 [==============================] - 1s 1s/step
...loading 0.08%...


KeyboardInterrupt: 

# Evaluate Predictions

In [ ]:
prediction_df = pd.read_csv("predictions_all_in_one.csv")
key = 'corrossion'

def count_numbers_in_ranges(numbers):
    range_counts = [0, 0, 0, 0]

    for num in numbers:
        if 0 <= num <= 0.165:
            range_counts[0] += 1
        elif 0.165 < num <= 0.5:
            range_counts[1] += 1
        elif 0.5 < num <= 0.825:
            range_counts[2] += 1
        elif 0.825 < num <= 1:
            range_counts[3] += 1

    return tuple(range_counts)


def all_in_one_confusion_matrix(labels_df, prediction_df):
    grouped = labels_df.groupby(key)['image_name'].agg(list)
    good_labels_dict = grouped.to_dict()
    good_pred_dict = {key: [] for key in good_labels_dict}
    print(good_pred_dict)

    for score in good_labels_dict:
        predictions = []

        for image_name in good_labels_dict[score]:
            row = prediction_df.loc[prediction_df['image_name'] == image_name]

            predictions.append(row[key].values[0])
            
        good_pred_dict[score] = count_numbers_in_ranges(predictions)
    
    return good_pred_dict



x = all_in_one_confusion_matrix(labels_df, prediction_df)
print(x)

{0.0: [], 0.33: [], 0.66: [], 1.0: []}
{0.0: (4057, 24, 1, 0), 0.33: (102, 300, 91, 3), 0.66: (18, 84, 236, 46), 1.0: (8, 1, 8, 315)}


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
%matplotlib qt

true_labels = []
predicted_labels = []

for true_class, counts in x.items():
    for pred_class, count in enumerate(counts):
        true_labels.extend([true_class] * count)
        predicted_labels.extend([pred_class] * count)

# Create a confusion matrix
truth = np.round(np.array(true_labels)/0.33)
conf_matrix = confusion_matrix(truth, predicted_labels)

# Calculate percentages along the rows
row_sums = conf_matrix.sum(axis=1, keepdims=True)
conf_matrix_percentages = conf_matrix / row_sums * 100

# Plot the confusion matrix using Seaborn
plt.figure(figsize=(10, 7))
heatmap = sns.heatmap(conf_matrix_percentages, annot=True, fmt=".1f", cmap="Blues",
            xticklabels=["0-0.165", "0.165-0.5", "0.5-0.825", "0.825-1"],
            yticklabels=["0.0", "0.33", "0.66", "1.0"])
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Corrosion Class Confusion Matrix')

cbar = heatmap.collections[0].colorbar
cbar.set_label('Percentage of Predictions (%)', rotation=90, labelpad=15)
plt.show()